# TrainClassifier

+ Faz o treinamento e avaliacao da acuracia dos classificadores utilizando um arquivo `.csv` gerado pelo notebook `Main.ipynb` 

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import emlearn
import os

In [2]:
# Carregar os dados de treinamento
try:
    training_data = pd.read_csv("../data/extracted_features/extracted_features_like_artigo_training.csv")
except FileNotFoundError:
    print("Arquivo de treinamento não encontrado. Crie um CSV de exemplo.")
    # Criando um DataFrame de exemplo para o código poder ser executado
    training_data = pd.DataFrame(np.random.rand(100, 15), columns=[f'feature_{i}' for i in range(14)] + ['FaultID'])
    training_data['FaultID'] = np.random.randint(low=0, high=5, size=100)


# Carregar os dados de teste
try:
    testing_data = pd.read_csv("../data/extracted_features/extracted_features_like_artigo_validation.csv")
except FileNotFoundError:
    print("Arquivo de teste não encontrado. Crie um CSV de exemplo.")
    # Criando um DataFrame de exemplo para o código poder ser executado
    testing_data = pd.DataFrame(np.random.rand(100, 15), columns=[f'feature_{i}' for i in range(14)] + ['FaultID'])
    testing_data['FaultID'] = np.random.randint(low=0, high=5, size=100)


# Definindo as colunas de features e o rótulo
# Ajuste os nomes das colunas conforme o seu arquivo CSV
feature_columns = ['RMS','Variance','Skewness','Kurtosis','CrestFactor','ShapeFactor','ImpulseFactor','MarginFactor','Peak1','Peak2','Peak3','PeakLocs1','PeakLocs2','PeakLocs3']
target_column = "FaultID"

# --- Embaralhar (shuffle) os dados ---
# O random_state=42 garante que o embaralhamento seja sempre o mesmo, para resultados reproduzíveis.
training_data = training_data.sample(frac=1, random_state=42).reset_index(drop=True)
testing_data = testing_data.sample(frac=1, random_state=42).reset_index(drop=True)
print("Dados de treinamento e teste foram embaralhados com sucesso.")

# Preparar os conjuntos de dados
X_train = training_data[feature_columns].astype('float32')
Y_train = training_data[target_column]

X_test = testing_data[feature_columns].astype('float32')
Y_test = testing_data[target_column]

print(f"Dados de treinamento carregados: {X_train.shape[0]} amostras")
print(f"Dados de teste carregados: {X_test.shape[0]} amostras")

Dados de treinamento e teste foram embaralhados com sucesso.
Dados de treinamento carregados: 5632 amostras
Dados de teste carregados: 1431 amostras


## Train Desktop Models
---

+ Treinamento indireto dos modelos semelhantes ao do artigo utilizando a funcionalidade do sklearn `cross_val_score`

In [3]:
# Dicionário com os classificadores para desktop
desktop_classifiers = {
    "Bagged Trees Ensemble": BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42),
    "Quadratic SVM": SVC(kernel='poly', degree=2, C=2),
    "Fine Decision Tree": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(n_neighbors=3)
}

print("--- Avaliação dos Modelos de Desktop ---")

# Treinar e avaliar cada classificador
results = {}
for name, model in desktop_classifiers.items():
    # Treinar o modelo
    model.fit(X_train, Y_train)
    
    # Fazer predições no conjunto de teste
    predictions = model.predict(X_test)
    
    # Calcular a acurácia
    accuracy = accuracy_score(Y_test, predictions)
    results[name] = accuracy
    print(f"Acurácia do {name}: {accuracy:.4f}")

--- Avaliação dos Modelos de Desktop ---
Acurácia do Bagged Trees Ensemble: 0.9783
Acurácia do Quadratic SVM: 0.6946
Acurácia do Fine Decision Tree: 0.9706
Acurácia do Gaussian Naive Bayes: 0.8931
Acurácia do K-Nearest Neighbors (KNN): 0.9245


## Training MCU Classifiers
---

In [4]:
classifiers_emlearn = {
    'decision_tree': DecisionTreeClassifier(),
    'random_forest': RandomForestClassifier(n_estimators=10, random_state=42),
    'extra_trees': ExtraTreesClassifier(n_estimators=10, random_state=42),
    'gaussian_naive_bayes': GaussianNB(),
    # 'knn' : KNeighborsClassifier(n_neighbors=3),
    # 'sklearn_mlp': sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10,10,), max_iter=1000, random_state=42)
}

In [5]:
def check_correctness(out_dir, name, model_filename, test_data, test_predictions, feature_columns):
    test_res = np.array(test_predictions).flatten()

    test_dataset = "\n".join([
        emlearn.cgen.array_declare(f"{name}_testset_data", dtype='float', values=test_data),
        emlearn.cgen.array_declare(f"{name}_testset_results", dtype='int', values=test_res),
        emlearn.cgen.constant_declare(f'{name}_testset_features', val=len(feature_columns)),
        emlearn.cgen.constant_declare(f'{name}_testset_samples', val=len(test_predictions)),
    ])

    test_code = test_dataset + \
    f'''
    #include "{model_filename}" // emlearn generated model

    #include <stdio.h> // printf

    int {name}_test() {{
        const int n_features = {name}_testset_features;
        const int n_testcases = {name}_testset_samples;

        int errors = 0;

        for (int i=0; i<n_testcases; i++) {{
            const float *features = {name}_testset_data + (i*n_features);
            const int expect_result = {name}_testset_results[i*1];

            const int out = model_predict(features, n_features);

            if (out != expect_result) {{
                printf(\"test-fail sample=%d expect=%d got=%d \\n\", i, expect_result, out);
                errors += 1;
            }}
            printf(\"test sample=%d expect=%d got=%d \\n\", i, expect_result, out);

        }}
        return errors;
    }}

    int main(int argc, const char *argv[])
    {{
        const int errors = {name}_test();
        printf(\"Errors: %d \\n\", errors);
        return errors;
    }}
    '''

    test_source_file = os.path.join(out_dir, f'test_{name}.c')
    with open(test_source_file, 'w') as f:
        f.write(test_code)

    print('Generated', test_source_file)
    print(f"Outdir: {out_dir}")
    include_dirs = [ emlearn.common.get_include_dir() ]
    test_executable = emlearn.common.compile_executable(
            test_source_file,
            out_dir,
            name=f'test_{name}',
            include_dirs=include_dirs
    )

    import subprocess
    errors = None
    try:
        print("TRY")
        subprocess.check_output(test_executable)
        errors = 0
        print("ERROR")
    except subprocess.CalledProcessError as e:
        errors = e.returncode
        print(f"CATCH {e.returncode}")

    return errors


In [6]:
def plot_results(ax, model, X, y):
    from sklearn.inspection import DecisionBoundaryDisplay

    # show classification boundaries
    DecisionBoundaryDisplay.from_estimator(
        model, X, alpha=0.4, ax=ax, response_method="auto",
    )

    # show datapoints
    ax.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, s=20, edgecolor="k")

In [7]:
def build_run_classifier(model, name, X_train, Y_train, X_test, Y_test, feature_columns):
    """
    Treina, avalia, converte e salva um classificador usando emlearn.
    """
    print(f"Processando o modelo: {name}")

    # Treina o modelo com o conjunto de treinamento
    model.fit(X_train, Y_train)
    
    # Avalia a acurácia no conjunto de teste
    accuracy = accuracy_score(Y_test, model.predict(X_test))
    print(f"Acurácia do modelo em Python: {accuracy:.4f}")

    # Converte o modelo treinado para código C
    inference_strategy = 'loadable'
    if type(model).__name__ in emlearn.trees.SUPPORTED_ESTIMATORS:
        inference_strategy = 'inline'

    cmodel = emlearn.convert(
        model,
        method=inference_strategy,
        dtype='float'
    )

    # Salva o modelo C em um arquivo .h
    out_dir = os.path.join(os.getcwd(), '../data/c_models')
    os.makedirs(out_dir, exist_ok=True)
    model_filename = os.path.join(out_dir, f'{name}_model.h')
    
    try:
        cmodel.save(file=model_filename, name='model')
        print(f"Modelo salvo em: {model_filename}")
    except Exception as e:
        print(f"Erro ao salvar o modelo {name}: {e}")

    # Verifica a acurácia do modelo convertido
    test_data_c = X_test.values.astype(np.float32)
    c_predictions = cmodel.predict(test_data_c)
    c_accuracy = accuracy_score(Y_test, c_predictions)
    print(f"Acurácia do modelo em C (emlearn): {c_accuracy:.4f}")

    return {
        'python_accuracy': accuracy,
        'c_accuracy': c_accuracy
    }

In [8]:
# Dicionário com os classificadores para emlearn
classifiers_emlearn = {
    'decision_tree': DecisionTreeClassifier(random_state=42),
    'random_forest': RandomForestClassifier(n_estimators=10, random_state=42),
    'extra_trees': ExtraTreesClassifier(n_estimators=10, random_state=42),
    'gaussian_naive_bayes': GaussianNB(),
}

print("\n--- Treinamento e Conversão dos Modelos para MCU ---")

mcu_results = {}
for name, model in classifiers_emlearn.items():
    result = build_run_classifier(model, name, X_train, Y_train, X_test, Y_test, feature_columns)
    mcu_results[name] = result
    print("----------------------------------------------------------------")


--- Treinamento e Conversão dos Modelos para MCU ---
Processando o modelo: decision_tree
Acurácia do modelo em Python: 0.9713
Modelo salvo em: /home/italolanza/Workspace/projeto-tg/python_code/notebooks/../data/c_models/decision_tree_model.h
Acurácia do modelo em C (emlearn): 0.9720
----------------------------------------------------------------
Processando o modelo: random_forest


tmp/myinlinetree_proba.c: In function ‘classify_proba’:
tmp/myinlinetree_proba.c:9:28: warning: unused variable ‘err’ [-Wunused-variable]
    9 |             const EmlError err = eml_trees_predict_proba(&myinlinetree, values, length, outputs, N_CLASSES);
      |                            ^~~


Acurácia do modelo em Python: 0.9804
Modelo salvo em: /home/italolanza/Workspace/projeto-tg/python_code/notebooks/../data/c_models/random_forest_model.h


tmp/myinlinetree_proba.c: In function ‘classify_proba’:
tmp/myinlinetree_proba.c:9:28: warning: unused variable ‘err’ [-Wunused-variable]
    9 |             const EmlError err = eml_trees_predict_proba(&myinlinetree, values, length, outputs, N_CLASSES);
      |                            ^~~


Acurácia do modelo em C (emlearn): 0.9804
----------------------------------------------------------------
Processando o modelo: extra_trees
Acurácia do modelo em Python: 0.9804
Modelo salvo em: /home/italolanza/Workspace/projeto-tg/python_code/notebooks/../data/c_models/extra_trees_model.h


tmp/myinlinetree_proba.c: In function ‘classify_proba’:
tmp/myinlinetree_proba.c:9:28: warning: unused variable ‘err’ [-Wunused-variable]
    9 |             const EmlError err = eml_trees_predict_proba(&myinlinetree, values, length, outputs, N_CLASSES);
      |                            ^~~


Acurácia do modelo em C (emlearn): 0.9804
----------------------------------------------------------------
Processando o modelo: gaussian_naive_bayes
Acurácia do modelo em Python: 0.8931
Modelo salvo em: /home/italolanza/Workspace/projeto-tg/python_code/notebooks/../data/c_models/gaussian_naive_bayes_model.h
Acurácia do modelo em C (emlearn): 0.7897
----------------------------------------------------------------


In [9]:
print("\n===== RESUMO FINAL DAS ACURÁCIAS =====\n")
print("--- Modelos de Desktop ---")
for name, acc in results.items():
    print(f"{name:<25} | Acurácia: {acc:.4f}")

print("\n--- Modelos para MCU (Python vs C) ---")
for name, res in mcu_results.items():
    print(f"{name:<25} | Acurácia Python: {res['python_accuracy']:.4f} | Acurácia C: {res['c_accuracy']:.4f}")


===== RESUMO FINAL DAS ACURÁCIAS =====

--- Modelos de Desktop ---
Bagged Trees Ensemble     | Acurácia: 0.9783
Quadratic SVM             | Acurácia: 0.6946
Fine Decision Tree        | Acurácia: 0.9706
Gaussian Naive Bayes      | Acurácia: 0.8931
K-Nearest Neighbors (KNN) | Acurácia: 0.9245

--- Modelos para MCU (Python vs C) ---
decision_tree             | Acurácia Python: 0.9713 | Acurácia C: 0.9720
random_forest             | Acurácia Python: 0.9804 | Acurácia C: 0.9804
extra_trees               | Acurácia Python: 0.9804 | Acurácia C: 0.9804
gaussian_naive_bayes      | Acurácia Python: 0.8931 | Acurácia C: 0.7897
